In [1]:
# Introducción documento "externo"
import PyPDF2

pdf_path = 'pdfs/Gatos.pdf'

# Abre el archivo PDF
with open(pdf_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    # Extrae texto de cada página
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    
# El texto completo del PDF ahora está en la variable 'text'
print(text)


Gatos
Los gatos son mamíferos carnívoros que pertenecen a la familia de los felinos. Son conocidos por
su agilidad, independencia y elegancia. Los gatos han sido compañeros de los humanos durante
miles de años.
Datos Curiosos:
- Los gatos pueden girar sus orejas 180 grados.
- Tienen un órgano especial llamado órgano de Jacobson que les permite saborear los olores.
- Los gatos domésticos comparten el 95.6% de su ADN con los tigres.
- Pueden dormir hasta 16 horas al día.
- El ronroneo de un gato tiene propiedades terapéuticas para los humanos.
- Los gatos ven en la oscuridad seis veces mejor que los humanos.
- Su nariz es tan única como una huella digital humana.
- Los bigotes de un gato son fundamentales para medir espacios y orientarse.
 
 
 
 
 
 
 
 
 
  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [2]:
# Split del texto recibido
import langchain

print(langchain.__version__)

from langchain.text_splitter  import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",  
    chunk_size=800,  
    # chunk_overlap=200 
)

splits = text_splitter.split_text(text)
print(len(splits))

0.3.11
2


In [3]:
# Vectorizar
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_texts(
    texts=splits,
    collection_name="ds_career",
    embedding=embeddings,
    persist_directory="./chroma_ds_career",
)

retriever = vector_store.as_retriever()

/home/bigdata/miniconda3/envs/rag_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
# Ollama 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)


In [ ]:
chain.invoke("¿Cuántas horas duerme un gato?")

'According to the provided context, if you want to progress in your data science career, Dr. Varshita Sher suggests that you need to develop three business skills:\n\n1. Collaboration-driving\n2. Project-scoping\n3. Stakeholder-managing\n4. Strategy-setting\n\nThe article includes resources for building those skills.'

In [ ]:
chain.invoke("¿Puede un gato girar las orejas 180 grados?")

'No, the text does not suggest that insulting one\'s boss would be an effective way to improve in a data science career. In fact, the text mentions "collaboration-driving", "project-scoping", "stakeholder-managing", and "strategy-setting" as skills that are important for progression in a data science career, which implies a more collaborative and professional approach.'

In [ ]:
chain.invoke("¿Qué es un gato?")

"Yes, according to the context, being a good communicator is a good idea. The text emphasizes that soft communication skills, such as data storytelling and cross-team collaboration, are crucial for managing teams, pitching products, communicating insights, motivating team members, negotiating better pay, securing funding, and accessing government grants. It suggests that developing these skills can open doors to new opportunities and improve one's career prospects as a data scientist transitioning into leadership roles."